<center>
    
## Project Type: Курсовой проект по курсу GeekBrains: Библиотеки Python для Data Science: продолжение
### Project Number: 2
# Project Title: Кредитный скоринг

#### Задача

Требуется, на основании имеющихся данных о клиентах банка, построить модель, используя обучающий датасет, для прогнозирования невыполнения долговых обязательств по текущему кредиту. Выполнить прогноз для примеров из тестового датасета.

#### Наименование файлов с данными

- course_project_train.csv - обучающий датасет
- course_project_test.csv - тестовый датасет

#### Целевая переменная

Credit Default - факт невыполнения кредитных обязательств

#### Метрика качества

F1-score (sklearn.metrics.f1_score)

#### Требования к решению

_Целевая метрика_

- F1 > 0.5
- Метрика оценивается по качеству прогноза для главного класса (1 - просрочка по кредиту)

### Import Libraries

---

In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Data Visualizations
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

### Import Data

---

In [6]:
data_path = 'C:/Users/rttri/Data Science Project/Курс GB - Библиотеки Python для Data Science продолжение/course_project_train.csv'

<center>
    
# Section 1.  Data Understanding

### _Collect initial data_

---

In [10]:
# Загрузим тренировочный набор даннх для первичного визуального анализа 
train_data = pd.read_csv(data_path)
train_data.head()

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
0,Own Home,482087.0,NaN,0.0,11.0,26.3,685960.0,1.0,NaN,1.0,debt consolidation,Short Term,99999999.0,47386.0,7914.0,749.0,0
1,Own Home,1025487.0,10+ years,0.0,15.0,15.3,1181730.0,0.0,NaN,0.0,debt consolidation,Long Term,264968.0,394972.0,18373.0,737.0,1
2,Home Mortgage,751412.0,8 years,0.0,11.0,35.0,1182434.0,0.0,NaN,0.0,debt consolidation,Short Term,99999999.0,308389.0,13651.0,742.0,0
3,Own Home,805068.0,6 years,0.0,8.0,22.5,147400.0,1.0,NaN,1.0,debt consolidation,Short Term,121396.0,95855.0,11338.0,694.0,0
4,Rent,776264.0,8 years,0.0,13.0,13.6,385836.0,1.0,NaN,0.0,debt consolidation,Short Term,125840.0,93309.0,7180.0,719.0,0


In [23]:
train_data.tail().T

,7495,7496,7497,7498,7499
Home Ownership,Rent,Home Mortgage,Rent,Home Mortgage,Rent
Annual Income,402192,1.53398e+06,1.87891e+06,NaN,NaN
Years in current job,< 1 year,1 year,6 years,NaN,4 years
Tax Liens,0,0,0,0,0
Number of Open Accounts,3,10,12,21,8
Years of Credit History,8.5,26.5,32.1,26.5,9.4
Maximum Open Credit,107866,686312,1.77892e+06,1.14125e+06,480832
Number of Credit Problems,0,0,0,0,0
Months since last delinquent,NaN,43,NaN,NaN,NaN
Bankruptcies,0,0,0,0,0


Первичный визуальный анализ показывает, что в тренировочном наборе имеются ряд категориальных перменных, рад колличественных перменных, пропущенные значения, нулевые значения, названия переменных содержат пробелы. 

Проведем дальнейшие начальные исследования набора данных. 

In [13]:
# Форма исходного набора данных
print("Форма исходного набора данных:\n{}".format(train_data.shape))

Форма исходного набора данных:
(7500, 17)


In [17]:
# Список столбцов
train_data.columns.tolist()

['Home Ownership',
 'Annual Income',
 'Years in current job',
 'Tax Liens',
 'Number of Open Accounts',
 'Years of Credit History',
 'Maximum Open Credit',
 'Number of Credit Problems',
 'Months since last delinquent',
 'Bankruptcies',
 'Purpose',
 'Term',
 'Current Loan Amount',
 'Current Credit Balance',
 'Monthly Debt',
 'Credit Score',
 'Credit Default']

In [18]:
# Анализ типов данных
train_data.dtypes 

Home Ownership                   object
Annual Income                   float64
Years in current job             object
Tax Liens                       float64
Number of Open Accounts         float64
Years of Credit History         float64
Maximum Open Credit             float64
Number of Credit Problems       float64
Months since last delinquent    float64
Bankruptcies                    float64
Purpose                          object
Term                             object
Current Loan Amount             float64
Current Credit Balance          float64
Monthly Debt                    float64
Credit Score                    float64
Credit Default                    int64
dtype: object

In [19]:
# Общая информация о наборе данных 
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Home Ownership                7500 non-null   object 
 1   Annual Income                 5943 non-null   float64
 2   Years in current job          7129 non-null   object 
 3   Tax Liens                     7500 non-null   float64
 4   Number of Open Accounts       7500 non-null   float64
 5   Years of Credit History       7500 non-null   float64
 6   Maximum Open Credit           7500 non-null   float64
 7   Number of Credit Problems     7500 non-null   float64
 8   Months since last delinquent  3419 non-null   float64
 9   Bankruptcies                  7486 non-null   float64
 10  Purpose                       7500 non-null   object 
 11  Term                          7500 non-null   object 
 12  Current Loan Amount           7500 non-null   float64
 13  Cur

### Выводы

- Набор имеет 17 переменных и 7500 наблюдений.
- В столбцах Annual Income, Years in current job, Months since last delinquent, Bankruptcies, Credit Score колличество наблюдений меньше, чем 7500. 
- Переменные Home Ownership, Years in current job, Purpose, Term имеют тип данных object.
- Переменная Credit Default имеет тип данных int64.
- Оставшиеся 12 переменных имеют тип данныз float64.

### _Describe data_

---

#### Описание датасета

1. Home Ownership - домовладение
1. Annual Income - годовой доход
1. Years in current job - количество лет на текущем месте работы
1. Tax Liens - налоговые обременения
1. Number of Open Accounts - количество открытых счетов
1. Years of Credit History - количество лет кредитной истории
1. Maximum Open Credit - наибольший открытый кредит
1. Number of Credit Problems - количество проблем с кредитом
1. Months since last delinquent - количество месяцев с последней просрочки платежа
1. Bankruptcies - банкротства
1. Purpose - цель кредита
1. Term - срок кредита
1. Current Loan Amount - текущая сумма кредита
1. Current Credit Balance - текущий кредитный баланс
1. Monthly Debt - ежемесячный долг
1. Credit Score -  кредитная оценка 
1. Credit Default - факт невыполнения кредитных обязательств (0 - погашен вовремя, 1 - просрочка)

#### Описание типов переменных

- Home Ownership - _категориальная перменная_ 
- Annual Income - **количественная переменная** 
- Years in current job - _категориальная перменная_
- Tax Liens - _категориальная перменная_
- Number of Open Accounts - **количественная переменная**
- Years of Credit History -  **количественная переменная**
- Maximum Open Credit - **количественная переменная**
- Number of Credit Problems - **количественная переменная** или _категориальная перменная_ - необходим дальнейший анализ 
- Months since last delinquent - **количественная переменная**
- Bankruptcies - банкротства - **количественная переменная** или _категориальная перменная_ - необходим дальнейший анализ 
- Purpose -  _категориальная перменная_
- Term - _категориальная перменная_
- Current Loan Amount - **количественная переменная**
- Current Credit Balance - **количественная переменная**
- Monthly Debt - **количественная переменная**
- Credit Score - **количественная переменная**
- Credit Default - _категориальная перменная_

**Итого**
- Категориальные переменные
    1. Home Ownership
    1. Years in current job
    1. Tax Liens
    1. Purpose
    1. Term
    1. Credit Default
- Количественные переменные
    1. Annual Income
    1. Number of Open Accounts
    1. Years of Credit History
    1. Maximum Open Credit
    1. Months since last delinquent
    1. Current Loan Amount
    1. Current Credit Balance
    1. Monthly Debt
    1. Credit Score
- Требуются дополнительные исследования 
    1. Number of Credit Problems
    1. Bankruptcies

### _Verify data quality_

---

In [24]:
def verify_data_quality (evaluated_dataset):
    # Проверка типов данных
    print("Типы данных:\n{}\n".format(evaluated_dataset.dtypes))
    # Проверка наличия значений NaN
    print("Пропущенные значения:\n{}\n".format(evaluated_dataset.isnull().sum().sum()))
    # Количество значений, отличных от NaN
    print("Количество значений, отличных от NaN:\n{}\n".format(evaluated_dataset.count().sum()))
    # Поиск дублирующих строк 
    print("Дублирующих строк:\n{}\n".format(evaluated_dataset.duplicated().sum()))
    # Форма нарбора данных
    print("Форма нарбора данных:\n{}\n".format(evaluated_dataset.shape))
    # Тип индекса набора данных
    print("Тип индекса набора данных:\n{}\n".format(evaluated_dataset.index))

In [26]:
# Оценка качетва данных
verify_data_quality(train_data)

Типы данных:
Home Ownership                   object
Annual Income                   float64
Years in current job             object
Tax Liens                       float64
Number of Open Accounts         float64
Years of Credit History         float64
Maximum Open Credit             float64
Number of Credit Problems       float64
Months since last delinquent    float64
Bankruptcies                    float64
Purpose                          object
Term                             object
Current Loan Amount             float64
Current Credit Balance          float64
Monthly Debt                    float64
Credit Score                    float64
Credit Default                    int64
dtype: object

Пропущенные значения:
7580

Количество значений, отличных от NaN:
119920

Дублирующих строк:
0

Форма нарбора данных:
(7500, 17)

Тип индекса набора данных:
RangeIndex(start=0, stop=7500, step=1)



### _Conclusions_

---

Тренировочный набор данных имеет:

- 6 катеориальных переменных
- 9 колличественных переменных 
- 2 переменные которые можно оценить как категориальные так и числовые, необходим дальнейший анализ
- Набор имеет 17 переменных и 7500 наблюдений.
- В столбцах Annual Income, Years in current job, Months since last delinquent, Bankruptcies, Credit Score колличество наблюдений меньше, чем 7500. 
- Переменные Home Ownership, Years in current job, Purpose, Term имеют тип данных object.
- Переменная Credit Default имеет тип данных int64.
- Оставшиеся 12 переменных имеют тип данныз float64.
-  7580 пропущенных значений 
- 119920 отличных от NaN
- Дублирующих строк нет 
- Индекс типа RangeIndex, непрерывный с шагом 1 

<center>

# Section 2. Data Preparation

## Clean data

---

### _Renaming_

---

### _Data type conversions_

---

### _Sorting, reordering, reindexing_

---

### _Addressing missing or invalid data_

---

### _Deduplicating data_

---

### _Filtering to the desired subset of data_

---

## Data Transformation

---

### _Concatenation_

---

### _Merging_

---

### _Pivoting_

---

### _Stacking & Unstacking_

---

### _Melting_

---

## Data enrichment

---

### _Adding new columns_

---

### _Binning_

---

### _Aggregating_

---

### _Resampling_

---

<center>

# Section 3.Conclusions